<a href="https://colab.research.google.com/github/ChanceDurr/DS-Unit-4-Sprint-2-Neural-Networks/blob/master/odule4-Hyperparameter-Tuning/Chance_Dare_LS_DS_434_Hyperparameter_Tuning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [0]:
import keras
import tensorflow
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
import wandb

In [2]:
pip install wandb

     |████████████████████████████████| 1.3MB 9.2MB/s 
     |████████████████████████████████| 92kB 28.7MB/s 
     |████████████████████████████████| 235kB 39.5MB/s 
     |████████████████████████████████| 81kB 25.0MB/s 
     |████████████████████████████████| 460kB 42.8MB/s 
     |████████████████████████████████| 102kB 29.5MB/s 
     |████████████████████████████████| 71kB 23.6MB/s 
     |████████████████████████████████| 256kB 47.6MB/s 
     |████████████████████████████████| 184kB 48.7MB/s 
  Created wheel for shortuuid: filename=shortuuid-0.5.0-cp36-none-any.whl size=5499 sha256=5feff43f68235e61e8f39cd8114aac0f42e5594f9aaafebae22b961be5051f1b
  Stored in directory: /root/.cache/pip/wheels/3f/eb/fd/69e5177f67b505e44acbd1aedfbe44b91768ee0c4cd5636576
  Created wheel for watchdog: filename=watchdog-0.9.0-cp36-none-any.whl size=73652 sha256=207377885290d1e79feff4fd286979fcfac420a47b9b68b6801a2e4b04147dbd
  Stored in directory: /root/.cache/pip/wheels/61/1d/d0/04cfe495619be2095eb8d89a31

In [2]:
!wandb login

You can find your API keys in your browser here: https://app.wandb.ai/authorize
Paste an API key from your profile: 432628d0c1492caac81ba34f619843052f94b04b
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/ChanceDurr/DS-Unit-4-Sprint-2-Neural-Networks/master/module4-Hyperparameter-Tuning/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [39]:
# # Create a Fresh Experiment
# wandb.init(project="hyperparameter_optimization_assignment", entity="ds5")

# # Configure said Fresh Experiment
# wandb.config.epochs = 50
# wandb.config.batch_size = 20
# wandb.config.hidden1_size = 20
# wandb.config.hidden1_function = 'relu'
# wandb.config.hidden2_size = 20
# wandb.config.hidden2_function = 'relu'
# wandb.config.hidden3_size = 20
# wandb.config.hidden3_function = 'sigmoid'



# Split Data into X and y
X = df[['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']]

y = df['Churn']


# Encode Data with LabelEncoder
encoder = LabelEncoder()

X_encoded = X.apply(encoder.fit_transform)
y_encoded = encoder.fit_transform(y)

# Add data that didn't need encoded
X_encoded['MonthlyCharges'] = df['MonthlyCharges']
X_encoded['TotalCharges'] = df['TotalCharges']

# Clean the TotalCharges column to replace missing values
X_encoded['TotalCharges'] = X_encoded['TotalCharges'].replace(' ', '0')
X_encoded['TotalCharges'] = X_encoded['TotalCharges'].astype('float')

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X_encoded,
                                                    y_encoded,
                                                    test_size=0.33)



# Create model wrapper
def create_model(n_neurons = 20):
  model = Sequential()

  # Add Input Layer
  model.add(Dense(n_neurons, activation='relu', input_shape=(X_train.shape[1],)))

  # Add Hidden Layers
  model.add(Dense(n_neurons, activation='relu'))
  model.add(Dense(n_neurons, activation='relu'))

  # Add Output Layer
  model.add(Dense(1, activation='sigmoid'))

  # Compile the model
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  
  return model

# Create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# Define the grid search parameters
param_grid = {'n_neurons': [15],
              'batch_size': [30]}

# Create Grid Search

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=2, cv=3)
grid_result = grid.fit(X_train, y_train)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

# Fit the model
# model.fit(X_train, y_train,
#           validation_data=(X_test, y_test),
#           epochs=wandb.config.epochs,
#           batch_size=wandb.config.batch_size,
#           callbacks=[WandbCallback()])


Best: 0.740144130409446 using {'batch_size': 30, 'n_neurons': 15}
Means: 0.740144130409446, Stdev: 0.0204998797939004 with: {'batch_size': 30, 'n_neurons': 15}
Means: 0.625900807630566, Stdev: 0.16096278109569526 with: {'batch_size': 30, 'n_neurons': 16}
Means: 0.6833404013164572, Stdev: 0.048568757698324895 with: {'batch_size': 30, 'n_neurons': 17}
Means: 0.6634167041274237, Stdev: 0.14434097993269468 with: {'batch_size': 30, 'n_neurons': 18}
Means: 0.739508266694569, Stdev: 0.011944822390407558 with: {'batch_size': 30, 'n_neurons': 19}


In [40]:
grid.best_estimator_.predict(X_test)

array([[0],
       [0],
       [0],
       ...,
       [1],
       [0],
       [0]])

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?